# 필요한 라이브러리 import

In [1]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf
import joblib
warnings.filterwarnings(action='ignore')

In [2]:
with open('./models/midfielder/midfielder_po.pickle','rb') as f:
    midfielder_po = pickle.load(f)

midfielder_model = load_model('./models/midfielder/midfielder_nn.h5')

premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [3]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [4]:
midfielder_df = premier_df[premier_df['포지션']=='미드필더']
midfielder_df = midfielder_df.reset_index(drop=True)

In [5]:
# Goal_Shoot_linear
midfielder_df['GSL'] = (midfielder_df['경기당 유효 슈팅/90']+0.5) + (midfielder_df['경기당 슈팅/90']) + (midfielder_df['골']*1.3)

In [6]:
# PSTL = Pass sucess try linear

midfielder_df['PSTL'] = (midfielder_df['경기당 패스 성공/90']) + (midfielder_df['경기당 패스 시도/90']*1.15)

In [7]:
independent_value ={'GSL','PSTL','공중볼시도/90','기회 창출/90','도움','드리블 성공/90','반칙 수/90','키패스/90','태클 성공/90'}

dependent_value = {'평균획득승점/경기'}

In [8]:
midfielder_df.loc[:,independent_value] = midfielder_po.transform(midfielder_df.loc[:,independent_value])

In [9]:
midfielder_df = midfielder_df.reset_index(drop=True)

In [10]:
midfielder_df['예측PPG']=midfielder_model.predict(midfielder_df.loc[:,independent_value])

In [11]:
midfielder_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,GSL,PSTL,예측PPG
46,Kevin De Bruyne,2021-2022,미드필더,맨시티,잉글랜드 프리미어 리그,11,108.9,0,33,11,...,47.70,-0.63,-0.4800,-3.602134,2.33,0.91,1.96,4.604651,165.222308,1.176509
112,Francisco Trincão,2021-2022,미드필더,울브스,잉글랜드 프리미어 리그,4,53.9,21/7/4 - 22/6/30,10,7,...,34.70,-0.65,-0.3625,-3.526327,5.35,7.27,0.97,1.065891,120.842308,1.146850
31,Trent Alexander-Arnold,2021-2022,미드필더,리버풀,잉글랜드 프리미어 리그,2,38.9,0,25,10,...,36.69,-0.70,-0.2075,-3.531407,5.97,9.22,1.85,-0.693798,127.743077,1.122473
2,Todd Cantwell,2021-2022,미드필더,노리치,잉글랜드 프리미어 리그,2,43.9,0,8,13,...,29.64,-0.55,-0.3225,-3.520466,3.72,5.53,0.92,-0.209302,105.273077,1.120898
50,João Cancelo,2021-2022,미드필더,맨시티,잉글랜드 프리미어 리그,1,28.9,0,19,7,...,34.75,-0.69,-0.0375,-3.465760,5.38,7.84,1.73,-1.248062,121.432308,1.089242


In [12]:
midfielder_min = np.min(midfielder_df['예측PPG'].tolist())
midfielder_min

1.0440709590911865

In [13]:
midfielder_df['WAR'] = midfielder_df['예측PPG'].map(lambda x : x-midfielder_min+1)

In [14]:
midfielder_df.loc[:,independent_value] = midfielder_po.inverse_transform(midfielder_df.loc[:,independent_value])

In [15]:
midfielder_war = midfielder_df.sort_values(by='WAR',ascending=False).copy()
midfielder_war = midfielder_war.reset_index(drop=True)

In [16]:
midfielder_war[['이름','키패스/90','기회 창출/90', '도움','WAR']].sort_values(by='WAR',ascending=False).head(10)

,이름,키패스/90,기회 창출/90,도움,WAR
0,Kevin De Bruyne,1.54,0.45,22.0,1.132438
1,Francisco Trincão,1.52,0.53,11.0,1.102779
2,Trent Alexander-Arnold,1.47,0.11,8.0,1.078402
3,Todd Cantwell,1.62,0.35,9.0,1.076827
4,João Cancelo,1.48,0.03,6.0,1.045171
5,Matt Lowton,0.96,0.51,5.0,1.041256
6,Saúl,1.43,0.52,14.0,1.017407
7,Moussa Sissoko,1.02,0.29,12.0,1.017268
8,Mads Roerslev,1.19,0.10,3.0,1.014241
9,Youri Tielemans,1.53,0.49,11.0,1.012775
